In [1]:
%load_ext autoreload
%autoreload 2

import helper
from helper import os as os

In [9]:
def driver_code():
    
    count = 0    
    file_path = '../../Data'
    
    for file in os.listdir(file_path):
        if file.startswith('applemobilitytrends'):
            file_path = os.path.join(file_path ,file)

    conn = helper._make_connection()
    helper.create_tables_apple(conn)

    df = helper.pd.read_csv(filepath_or_buffer=file_path, low_memory=False)

    dates = df[df.columns[6:]]
    nulls = dates[dates.isna().all(axis=1)].index
    df.drop(index=nulls, inplace=True)

    countries_util = df[['geo_type', 'region', 'alternative_name', 'sub-region', 'country']].drop_duplicates()

    countries_util = countries_util[['region', 'geo_type', 'alternative_name', 'sub-region', 'country']]

    df = df.drop(labels=['geo_type', 'alternative_name', 'country', 'sub-region'], axis=1)
    df.head()

    inter_df = helper.rearrange_df(df)

    count += helper.import_data(conn, 'countries_apple', countries_util)
    count += helper.import_data(conn, 'mobility_stats_apple', inter_df)
    
    helper.os.remove(file_path)
    
    return count, file_path

In [10]:
count, file_path = driver_code()
print(count)

Connecting to the PostgreSQL database...
Connection successful
Version: ('PostgreSQL 14.0, compiled by Visual C++ build 1914, 64-bit',)
Fetching Data from database...
Tables created successfully!
----------------------------
Truncated countries_apple
Done!
-------------------------------
Truncated mobility_stats_apple
Done!
-------------------------------
2
